In [1]:
import pandas as pd
import os, json
import matplotlib.pyplot as plt
import numpy as np

# Merge raw metadata

In [399]:
def parse_xlsx(n, root='/home/jhyang/WORKSPACES/DATA/2D/pl_postech/spectrum/', version=1):
    if not isinstance(n, str):
        n = str(n)
    if not n.endswith('.xlsx'):
        n += '.xlsx'
    fn = os.path.join(root, n)
    if not os.path.isfile(fn):
        return None
    data_id = n.replace('.xlsx','')
    df1 = pd.read_excel(fn, header=1, 
                    names=['Classification','Item','Details','Value','Unit','Remark'], 
                    usecols=[1,2,3,4,5,6])#, index_col=[0,1,2])
    df2 = pd.read_excel(fn, header=1, 
                    names=['Classification','Item','Details','Value','Unit','Remark'], 
                    usecols=[1,2,3,4,5,6], index_col=[0,1,2])

    count = 0
    for i, row in df1.iterrows():
        isBreak = row.isna().values.sum() == 6
        if isBreak:
            count += 1
        else:
            count = 0
        if count == 2:
            break
    df = df2.iloc[:i-1].copy()
    if version == 1:
        df.set_index(['Unit'], inplace=True, append=True)
        df.loc[('Sample', 'ID', np.NaN, np.NaN)] = [np.NaN, np.NaN]
    elif version == 2:
        part_t = df.T
        idx = part_t.pop(('Sample', 'Sample ID',np.NaN)).values
        volt = part_t.pop(('Measure','PL','Voltage')).values
        temp = part_t.pop(('Measure','PL','Temperature')).values
        atemp = part_t.pop(('Sample','Annealing','Temperature')).values

        part_t[('Sample','ID',np.NaN)] = idx
        part_t[('Measure','Gate voltage',np.NaN)] = volt
        if temp[1] == '℃' or 'C' in temp[1]:
            temp[0] += 273.15
            temp[1] = 'K'
        if atemp[1] == 'K':
            if 'nan' not in str(atemp[0]).lower():
                atemp[0] = atemp[0] - 273.15
        atemp[1] = 'degree'
        part_t[('Measure','Temperature',np.NaN)] = temp
        part_t[('Sample','Annealing','Temperature')] = atemp
        df = part_t.T
        df.set_index(['Unit'], inplace=True, append=True)
    df = df.sort_index()
    if isinstance(df.index, pd.MultiIndex):
        df.index = pd.MultiIndex.from_frame(
            df.index.to_frame().fillna('N/A')
        )
    else:
        df.index = df.index.fillna('N/A')
    
    val = df['Value']
    rem = df['Remark']
    val.name = data_id
    rem.name = data_id
    return val.T, rem.T

In [400]:
idxs = [765, 766, 767, 768, 769, 772, 773, 774, 775, 776, 777, 778, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788, 789, 790, 791, 792] 
new_df = None
for idx in idxs:
    a, b = parse_xlsx(idx, version=2)
    a = a.to_frame().T
    if new_df is None:
        new_df = a.copy()
    else:
        new_df = pd.concat([new_df, a])

In [401]:
new_df

Classification           Fitting                                       \
Item                          PL                                Raman   
Details        center wavelength cut-off count width center frequency   
Unit                          nm           N/A    nm             cm-1   
765                          640           740   NaN              NaN   
766                          640           740   NaN              NaN   
767                          640           740   NaN              NaN   
768                          640           740   NaN              NaN   
769                          640           740   NaN              NaN   
772                          640           740   NaN              NaN   
773                          640           740   NaN              NaN   
774                          760             0   NaN              NaN   
775                          760             0   NaN              NaN   
776                          760             0   NaN              NaN   
777                          760             0   NaN              NaN   
778                          760             0   NaN              NaN   
779                          760             0   NaN              NaN   
780                          760             0   NaN              NaN   
781                          760             0   NaN              NaN   
782                          850             0   NaN              NaN   
783                          860             0   NaN              NaN   
784                          860             0   NaN              NaN   
785                          860             0   NaN              NaN   
786                          860             0   NaN              NaN   
787                          860             0   NaN              NaN   
788                          860             0   NaN              NaN   
789                          880           575   NaN              NaN   
790                          850             0   NaN              NaN   
791                          670             0   NaN              NaN   
792                          670             0   NaN              NaN   

Classification                        Measure                        \
Item                                     Date Excitation Laser        
Details        cut-off count width        N/A            Power Type   
Unit                     N/A  cm-1        N/A               uW  N/A   
765                      NaN   NaN 2022-10-12              100   CW   
766                      NaN   NaN 2022-10-12              100   CW   
767                      NaN   NaN 2022-10-12              100   CW   
768                      NaN   NaN 2022-10-12              100   CW   
769                      NaN   NaN 2022-10-12              100   CW   
772                      NaN   NaN 2022-10-14                1   CW   
773                      NaN   NaN 2022-10-14                1   CW   
774                      NaN   NaN 2022-11-28              100   CW   
775                      NaN   NaN 2022-11-28              100   CW   
776                      NaN   NaN 2022-11-28              100   CW   
777                      NaN   NaN 2022-11-28              100   CW   
778                      NaN   NaN 2022-11-28              100   CW   
779                      NaN   NaN 2022-11-28              100   CW   
780                      NaN   NaN 2022-11-28              100   CW   
781                      NaN   NaN 2022-11-28              100   CW   
782                      NaN   NaN 2022-08-20               50   CW   
783                      NaN   NaN 2022-09-22              200   CW   
784                      NaN   NaN 2022-09-22              200   CW   
785                      NaN   NaN 2022-09-22              200   CW   
786                      NaN   NaN 2022-12-05              200   CW   
787                      NaN   NaN 2022-12-05              200   CW   
788                      NaN   NaN

In [402]:
mapper = {
    'CVD growth': 'CVD',
    'CVD': 'CVD',
    'Exfoliation': 'Exfoliation',
    'exfoliation': 'Exfoliation',
}

In [407]:
old_df = pd.read_csv('/mnt/DATA/2D/pl_postech/datalist_update_20221130.csv', header=[0,1,2,3], index_col=0).T
old_df.index = pd.MultiIndex.from_frame(
    old_df.index.to_frame().applymap(lambda x: 'N/A' if x == 'nan' else x)
)
#old_df.T[:34].T.to_csv('/mnt/DATA/2D/pl_postech/datalist_update_20221130.csv')
df = pd.concat([old_df[:33].T, new_df])
df = df.applymap(lambda x: mapper[x] if x in mapper.keys() else x)
df.to_csv('/mnt/DATA/2D/pl_postech/datalist_update_20230626.csv')

# Saving metdata

In [417]:
df.Sample.Substrate

Details,N/A
Unit,N/A
100,SiO2/Si
101,SiO2/Si
125,SiO2/Si
126,SiO2/Si
128,SiO2/Si
...,...
788,SiO2/Si
789,SiO2/Si
790,TEM grid


In [2]:
#df = pd.read_csv('/mnt/DATA/2D/pl_postech/datalist_update_20221130.csv', header=[0,1,2,3], index_col=0)
df = pd.read_csv('/mnt/DATA/2D/pl_postech/datalist_update_20230626.csv', header=[0,1,2,3], index_col=0)

col_pl  = ('Data','Type','PL','bool')
col_lp  = ('Measure', 'Excitation Laser', 'Power', 'uW')
col_lw  = ('Measure', 'Excitation Laser', 'Wavelength', 'nm')
col_it  = ('Measure', 'Mapping condition', 'Integration Time', 's')
col_t   = ('Measure', 'Temperature', 'N/A', 'K')
col_cem = ('Sample',  'TMD composition', 'N/A', 'N/A')
col_met = ('Sample',  'Method', 'N/A', 'N/A')
col_sub = ('Sample',  'Substrate', 'N/A', 'N/A')
col_sid = ('Sample',  'ID', 'N/A', 'N/A')
col_res = ('Measure', 'Mapping condition', 'Data point', 'N/A')
col_size = ('Measure', 'Mapping condition', 'Size', 'um')
col_cen_pl = ('PL','center wavelength','nm')
col_cen_rm = ('Raman','center frequency','cm-1')

cens = {}
params = df.copy()
params.pop(('Measure','Date'))
fits = params.pop(('Fitting'))
pls  = fits[col_cen_rm].isna()
res = params.pop(col_res)

/tmp/ipykernel_55545/2910043190.py:20: PerformanceWarning: indexing past lexsort depth may impact performance.
  params.pop(('Measure','Date'))


In [3]:
features = {
    'PL':[], # is PL?
    'LP':[], # Excitation Laser Power in uW
    'LW':[], # Excitation Laser Wavelength in nm
    'IT':[], # Integration time in s
    'T':[], # Measure temperature
    'CEM':[], # Chemical composition
    'SUB':[], # substrate
    'MET':[], # method
    'RES1':[], # Resolution of input
    'RES2':[], # Resolution of output
    'IDX1':[], # Index of input
    'IDX2':[], # Index of output
    'SIZEx':[],
    'SIZEy':[],
    'SID':[], # sample ID
}
js = []
for i, param_i in params.iterrows():

    cem = param_i[col_cem]
    ri  = res.loc[i]
    sx  = param_i[col_size]
    if '*' in sx:
        sx, sy = sx.split('*')
    else:
        sx, sy = sx, sx
    if cem not in cens.keys():
        cens[cem] = []
    for j, param_j in params.iterrows():
        if j <= i: continue
        if not param_i.equals(param_j): continue
        rj = res.loc[j]
        if ri == rj: continue
        if ri > rj:
            i1, r1, i2, r2 = j, rj, i, ri
        else:
            i1, r1, i2, r2 = i, ri, j, rj

        features['RES1'].append(r1)
        features['RES2'].append(r2)
        features['IDX1'].append(i1)
        features['IDX2'].append(i2)
        features['PL'].append(int(pls.loc[i]))
        features['LP'].append(param_i[col_lp])
        features['LW'].append(param_i[col_lw])
        features['IT'].append(param_i[col_it])
        features['T'].append(param_i[col_t])
        features['CEM'].append(cem)
        features['SID'].append(param_i[col_sid])
        features['SUB'].append(param_i[col_sub])
        features['MET'].append(param_i[col_met])
        features['SIZEx'].append(sx)
        features['SIZEy'].append(sy)

In [4]:
srdf = pd.DataFrame(features)
#srdf.to_csv('/mnt/DATA/2D/pl_postech/super_resolution_metadata_20230626.csv', index=False)
srdf

,PL,LP,LW,IT,T,CEM,SUB,MET,RES1,RES2,IDX1,IDX2,SIZEx,SIZEy,SID
0,0,2000,532,0.025,300.00,WS2,SiO2/Si,CVD,15,18,302,305,15,15,44
1,0,2000,532,0.025,300.00,WS2,SiO2/Si,CVD,15,20,302,308,15,15,44
2,0,2000,532,0.025,300.00,WS2,SiO2/Si,CVD,15,23,302,311,15,15,44
3,0,2000,532,0.025,300.00,WS2,SiO2/Si,CVD,15,25,302,314,15,15,44
4,0,2000,532,0.050,300.00,WS2,SiO2/Si,CVD,15,18,303,306,15,15,44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,1,100,632,1.000,300.15,WSe2,SiO2/Si,CVD,10,40,778,780,20,20,94
433,1,100,632,1.000,300.15,WSe2,SiO2/Si,CVD,10,80,778,781,20,20,94
434,1,100,632,1.000,300.15,WSe2,SiO2/Si,CVD,20,40,779,780,20,20,94
435,1,100,632,1.000,300.15,WSe2,SiO2/Si,CVD,20,80,779,781,20,20,94


In [5]:
features_ = {
    'PL':[], # is PL?
    'LP':[], # Excitation Laser Power in uW
    'LW':[], # Excitation Laser Wavelength in nm
    'IT':[], # Integration time in s
    'T':[], # Measure temperature
    'CEM':[], # Chemical composition
    'SUB':[], # substrate
    'MET':[], # method
    'RES':[], # Resolution of input
    'IDX':[], # Index of input
    'SIZEx':[],
    'SIZEy':[],
    'SID':[], # sample ID
}

col_pl  = ('Data','Type','PL','bool')
col_lp  = ('Measure', 'Excitation Laser', 'Power', 'uW')
col_lw  = ('Measure', 'Excitation Laser', 'Wavelength', 'nm')
col_it  = ('Measure', 'Mapping condition', 'Integration Time', 's')
col_t   = ('Measure', 'Temperature', 'N/A', 'K')
col_cem = ('Sample',  'TMD composition', 'N/A', 'N/A')
col_met = ('Sample',  'Method', 'N/A', 'N/A')
col_sub = ('Sample',  'Substrate', 'N/A', 'N/A')
col_sid = ('Sample',  'ID', 'N/A', 'N/A')
col_res = ('Measure', 'Mapping condition', 'Data point', 'N/A')
col_size = ('Measure', 'Mapping condition', 'Size', 'um')
col_cen_pl = ('PL','center wavelength','nm')
col_cen_rm = ('Raman','center frequency','cm-1')

cens = {}
params = df.copy()
params.pop(('Measure','Date'))
fits = params.pop(('Fitting'))
pls  = fits[col_cen_rm].isna()
res = params.pop(col_res)

js = []
for i, param_i in params.iterrows():

    cem = param_i[col_cem]
    sx  = param_i[col_size]
    if '*' in sx:
        sx, sy = sx.split('*')
    else:
        sx, sy = sx, sx

    features_['RES'].append(res.loc[i])
    features_['IDX'].append(i)
    features_['PL'].append(int(pls.loc[i]))
    features_['LP'].append(param_i[col_lp])
    features_['LW'].append(param_i[col_lw])
    features_['IT'].append(param_i[col_it])
    features_['T'].append(param_i[col_t])
    features_['CEM'].append(cem)
    features_['SID'].append(param_i[col_sid])
    features_['SUB'].append(param_i[col_sub])
    features_['MET'].append(param_i[col_met])
    features_['SIZEx'].append(sx)
    features_['SIZEy'].append(sy)

/tmp/ipykernel_55545/359191226.py:33: PerformanceWarning: indexing past lexsort depth may impact performance.
  params.pop(('Measure','Date'))


In [38]:
pd.set_option('display.max_rows',75)
rdf = pd.DataFrame(features_)
rdf[(rdf.RES > 25) & (rdf.RES < 40)]
#rdf[(rdf.RES < 50) & (rdf.RES > 10) & (rdf.CEM == 'WSe2/WS2') & (rdf.LP < 150) & (rdf.PL == 1)]
#rdf[(rdf.CEM == 'WSe2/WS2')]

,PL,LP,LW,IT,T,CEM,SUB,MET,RES,IDX,SIZEx,SIZEy,SID
2,1,5,532,1.0,300.0,WSe2,SiO2/Si,Exfoliation,30,125,15,15,3
3,0,1500,532,1.0,300.0,grapehen,SiO2/Si,Exfoliation,38,126,19,19,4


In [51]:
rdf[rdf.SID == '5']


,PL,LP,LW,IT,T,CEM,SUB,MET,RES,IDX,SIZEx,SIZEy,SID
4,1,50,532,0.1,300.0,WS2,SiO2/Si,CVD,60,128,20,20,5


# Check data

In [2]:
srdf = pd.read_csv('x:/DATA/2D/pl_postech/super_resolution_metadata_20221025.csv')
sids = srdf.SID.drop_duplicates().values
sid = sids[10]

part = srdf[srdf.SID == sid]
print(sorted(set(np.hstack([part.IDX1.values, part.IDX2.values]))))
spart = part[:0]
for res in [2,4]:
    mask = part.RES2 / part.RES1 == res
    for _, row in part[mask].iterrows():
        print('[', ', '.join([v for v in row[['IDX1','IDX2']].values.astype(str)]), '],')
    spart = pd.concat([spart, part[mask]])
    print()
spart

[413, 414]
[ 413, 414 ],




,PL,LP,LW,IT,T,CEM,RES1,RES2,IDX1,IDX2,SID
143,1,10,632,5.0,77.0,MoSe2,20,40,413,414,57


In [24]:
sids = [79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89]
mask = None
for sid in sids:
    if mask is None:
        mask = (srdf.SID == sid).values
    else:
        mask = mask | (srdf.SID == sid).values
part = srdf.loc[mask]
res = part.RES2 / part.RES1
part[res == 8].shape

(22, 11)

In [79]:
df = pd.read_csv('/mnt/DATA/2D/pl_postech/datalist_update_20230626.csv', header=[0,1,2,3])
df.columns

MultiIndex([(     'Classification',                'Item', ...),
            (            'Fitting',                  'PL', ...),
            (            'Fitting',                  'PL', ...),
            (            'Fitting',                  'PL', ...),
            (            'Fitting',               'Raman', ...),
            (            'Fitting',               'Raman', ...),
            (            'Fitting',               'Raman', ...),
            (            'Measure',                'Date', ...),
            (            'Measure',    'Excitation Laser', ...),
            (            'Measure',    'Excitation Laser', ...),
            (            'Measure',    'Excitation Laser', ...),
            (            'Measure',   'Mapping condition', ...),
            (            'Measure',   'Mapping condition', ...),
            (            'Measure',   'Mapping condition', ...),
            (            'Measure',         'Temperature', ...),
            (            